<a href="https://colab.research.google.com/github/CodeHunterOfficial/ABC_DataMining/blob/main/NLP/NLP-2025/%D0%A2%D0%BE%D0%BA%D0%B5%D0%BD%D0%B8%D0%B7%D0%B0%D1%82%D0%BE%D1%80%D1%8B%20(%D0%BF%D1%80%D0%B8%D0%BC%D0%B5%D1%80%D1%8B)_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BPE

In [ ]:
import collections

class BPE:
    def __init__(self, vocab_size):
        """
        Инициализация BPE токенизатора.
        :param vocab_size: Желаемый размер словаря.
        """
        self.vocab_size = vocab_size
        self.vocab = {}  # Словарь подслов
        self.merges = [] # Список выполненных слияний
        self.token_to_id = {} # Отображение токенов на их ID
        self.id_to_token = {} # Отображение ID на токены

    def _get_pairs(self, word_tokens):
        """
        Находит все смежные пары токенов в списке токенов слова.
        :param word_tokens: Список токенов слова (например, ['l', 'o', 'w']).
        :return: Словарь, где ключ - это пара токенов, значение - ее частота.
        """
        pairs = collections.defaultdict(int)
        for i in range(len(word_tokens) - 1):
            pairs[(word_tokens[i], word_tokens[i+1])] += 1
        return pairs

    def train(self, corpus):
        """
        Обучает BPE токенизатор на заданном корпусе.
        :param corpus: Список строк (предложений) для обучения.
        """
        # Шаг 1: Инициализация - каждое слово разбивается на символы
        # Добавляем символ '_' для обозначения пробелов, как в SentencePiece
        word_freqs = collections.defaultdict(int)
        for text in corpus:
            # Заменяем пробелы на специальный символ и разбиваем на слова
            # Затем каждое "слово" разбиваем на символы
            processed_text = text.replace(' ', '_')
            words = processed_text.split('_') # Временное разбиение для обработки

            for word in words:
                if word: # Убедимся, что слово не пустое
                    word_freqs[tuple(list(word))] += 1 # Словарь: (символы слова) -> частота

        # Инициализируем словарь всеми уникальными символами
        initial_vocab = set()
        for word_tokens in word_freqs.keys():
            initial_vocab.update(word_tokens)

        # Добавляем символ пробела в начальный словарь
        initial_vocab.add('_')

        # Преобразуем начальный словарь в список для присвоения ID
        self.vocab = {token: idx for idx, token in enumerate(sorted(list(initial_vocab)))}
        self.token_to_id = {token: idx for idx, token in enumerate(sorted(list(initial_vocab)))}
        self.id_to_token = {idx: token for idx, token in enumerate(sorted(list(initial_vocab)))}

        print(f"Начальный размер словаря: {len(self.vocab)}")
        print(f"Начальные токены: {list(self.vocab.keys())[:20]}...") # Показываем первые 20

        # Шаг 2: Итеративное слияние пар
        while len(self.vocab) < self.vocab_size:
            all_pairs = collections.defaultdict(int)
            # Для каждого слова в корпусе, представленного его текущими токенами
            for word_tokens, freq in word_freqs.items():
                pairs = self._get_pairs(list(word_tokens))
                for pair, count in pairs.items():
                    all_pairs[pair] += count * freq # Учитываем частоту слова

            if not all_pairs:
                break # Если нет больше пар для слияния, выходим

            # Находим наиболее частую пару
            best_pair = max(all_pairs, key=all_pairs.get)

            # Если частота лучшей пары равна 1, и мы не можем достичь vocab_size,
            # или если мы уже достигли максимального количества слияний,
            # или если нет больше уникальных пар для добавления,
            # можно выйти из цикла.
            if all_pairs[best_pair] == 1 and len(self.vocab) >= self.vocab_size:
                break

            new_token = "".join(best_pair)

            # Если новый токен уже в словаре, пропускаем
            if new_token in self.vocab:
                continue

            # Добавляем новый токен в словарь
            new_id = len(self.vocab)
            self.vocab[new_token] = new_id
            self.token_to_id[new_token] = new_id
            self.id_to_token[new_id] = new_token
            self.merges.append(best_pair) # Записываем выполненное слияние

            # Обновляем токены в word_freqs
            new_word_freqs = collections.defaultdict(int)
            for word_tokens, freq in word_freqs.items():
                # Заменяем все вхождения best_pair на new_token в word_tokens
                merged_tokens = self._merge_pair_in_list(list(word_tokens), best_pair, new_token)
                new_word_freqs[tuple(merged_tokens)] += freq
            word_freqs = new_word_freqs

            # print(f"Слияние: {best_pair} -> {new_token}. Новый размер словаря: {len(self.vocab)}")

        print(f"\nОбучение завершено. Итоговый размер словаря: {len(self.vocab)}")
        print(f"Примеры токенов после обучения: {list(self.vocab.keys())[len(initial_vocab):len(initial_vocab)+20]}...")


    def _merge_pair_in_list(self, tokens, pair, new_token):
        """
        Вспомогательная функция для слияния пары в списке токенов.
        """
        result = []
        i = 0
        while i < len(tokens):
            if i + 1 < len(tokens) and (tokens[i], tokens[i+1]) == pair:
                result.append(new_token)
                i += 2
            else:
                result.append(tokens[i])
                i += 1
        return result

    def tokenize(self, text):
        """
        Токенизирует входной текст, используя обученную модель BPE.
        :param text: Входная строка.
        :return: Список подслов (строк).
        """
        # Предварительная обработка текста: замена пробелов на '_'
        processed_text = text.replace(' ', '_')

        # Начальная токенизация на символы
        tokens = list(processed_text)

        # Применяем все выполненные слияния в том же порядке
        for pair, new_token in self.merges:
            tokens = self._merge_pair_in_list(tokens, pair, new_token)

        # Финальная проверка: если какой-то токен не был объединен,
        # убедимся, что он есть в словаре. Если нет, это OOV символ,
        # но в нашей реализации BPE все символы из обучающего корпуса
        # должны быть в начальном словаре.

        # Для удобства вывода, если токен начинается с '_',
        # это означает начало слова.
        final_pieces = []
        current_piece = ""
        for token in tokens:
            if token.startswith('_') and current_piece:
                final_pieces.append(current_piece)
                current_piece = token
            else:
                current_piece += token
        if current_piece:
            final_pieces.append(current_piece)

        return final_pieces

    def decode(self, tokens):
        """
        Декодирует список подслов обратно в исходный текст.
        :param tokens: Список подслов (строк).
        :return: Восстановленная строка.
        """
        # Объединяем токены, заменяя '_ ' на ' '
        # Сначала объединяем все токены
        merged_text = "".join(tokens)
        # Затем заменяем специальный символ пробела на обычный
        decoded_text = merged_text.replace('_', ' ').strip() # strip() для удаления начального/конечного пробела
        return decoded_text

# --- Пример использования ---
print("\n--- Демонстрация работы BPE с нуля ---")

# Обучающий корпус
corpus = [
    "Учитель говорит: «Откройте учебники».",
    "Ученики открывают учебники.",
    "Откройте страницу 15.",
    "Страница сложная.",
    "Страницы учебника содержат много информации."
]

# Создаем и обучаем BPE токенизатор
# Зададим небольшой размер словаря для наглядности процесса слияния.
# В реальных задачах vocab_size может быть 8000, 16000, 32000 и т.д.
bpe_tokenizer = BPE(vocab_size=50) # Попробуйте изменить vocab_size
bpe_tokenizer.train(corpus)

# --- Токенизация ---
print("\n--- Токенизация нового текста ---")
text_to_tokenize = "Учитель говорит: «Откройте учебники». Ученики открывают учебники. Откройте страницу 15. Страница сложная. Страницы учебника содержат много информации."

# Токенизируем текст
tokenized_pieces = bpe_tokenizer.tokenize(text_to_tokenize)
print(f"Исходный текст: '{text_to_tokenize}'")
print(f"Токенизированные подслова: {tokenized_pieces}")

# --- Детокенизация ---
print("\n--- Детокенизация ---")
decoded_text = bpe_tokenizer.decode(tokenized_pieces)
print(f"Детокенизированный текст: '{decoded_text}'")

# Проверка обратимости
if text_to_tokenize.replace(' ', '') == decoded_text.replace(' ', ''): # Сравниваем без пробелов для упрощения
    print("\nПроцесс токенизации и детокенизации успешно обратим (без учета оригинальных пробелов).")
else:
    print("\nВнимание: Детокенизированный текст не полностью совпадает с исходным.")
    print(f"Ожидалось (без пробелов): '{text_to_tokenize.replace(' ', '')}'")
    print(f"Получено (без пробелов): '{decoded_text.replace(' ', '')}'")

print("\n--- Демонстрация работы BPE завершена ---")


#WordPiece

In [ ]:
import collections

def preprocess_text(text):
    """
    Предварительная обработка текста: приведение к нижнему регистру и разбиение на слова.
    """
    # Удаляем знаки препинания и приводим к нижнему регистру
    processed_text = text.lower().replace('...', '').replace('.', '')
    # Разбиваем на слова по пробелам
    words = processed_text.split(' ')
    # Добавляем пробелы как отдельные токены между словами
    initial_tokens = []
    for i, word in enumerate(words):
        if word: # Убедимся, что слово не пустое
            initial_tokens.extend(list(word))
        if i < len(words) - 1:
            initial_tokens.append(' ') # Добавляем пробел между словами
    return initial_tokens, words # Возвращаем начальные токены и список слов для удобства

def calculate_frequencies(tokens):
    """
    Подсчет частот отдельных токенов (униграмм) и биграмм в текущем корпусе.
    """
    unigram_freq = collections.defaultdict(int)
    bigram_freq = collections.defaultdict(int)

    for i in range(len(tokens)):
        unigram_freq[tokens[i]] += 1
        if i < len(tokens) - 1:
            bigram_freq[(tokens[i], tokens[i+1])] += 1
    return unigram_freq, bigram_freq

def calculate_score(unigram_freq, bigram_freq):
    """
    Вычисление оценки слияния для всех возможных биграмм.
    Score(A, B) = frequency(AB) / (frequency(A) * frequency(B))
    """
    scores = {}
    for (token_a, token_b), freq_ab in bigram_freq.items():
        freq_a = unigram_freq[token_a]
        freq_b = unigram_freq[token_b]
        if freq_a > 0 and freq_b > 0: # Избегаем деления на ноль
            score = freq_ab / (freq_a * freq_b)
            scores[(token_a, token_b)] = score
    return scores

def merge_tokens(tokens, best_bigram):
    """
    Слияние лучшей биграммы в корпусе.
    """
    merged_token = "".join(best_bigram)
    new_tokens = []
    i = 0
    while i < len(tokens):
        if i + 1 < len(tokens) and (tokens[i], tokens[i+1]) == best_bigram:
            new_tokens.append(merged_token)
            i += 2 # Пропускаем оба слитых токена
        else:
            new_tokens.append(tokens[i])
            i += 1
    return new_tokens

def train_wordpiece(text, target_vocab_size=None, num_merges=None):
    """
    Обучение WordPiece токенизатора.
    Итеративно сливает токены до достижения целевого размера словаря
    или заданного количества слияний.
    """
    print(f"Исходный текст: '{text}'")
    current_tokens, _ = preprocess_text(text)

    # Начальный словарь состоит из всех уникальных символов
    vocab = set(current_tokens)
    print(f"\nШаг 1: Начальный словарь (V0) содержит {len(vocab)} токенов.")
    print(f"Начальный корпус: {current_tokens}")

    merges_count = 0
    while True:
        unigram_freq, bigram_freq = calculate_frequencies(current_tokens)
        scores = calculate_score(unigram_freq, bigram_freq)

        if not scores:
            print("\nНет больше биграмм для слияния. Остановка.")
            break

        # Находим биграмму с наивысшей оценкой
        best_bigram = max(scores, key=scores.get)
        best_score = scores[best_bigram]

        # Критерий остановки: если оценка слишком низкая (можно задать порог)
        # Для простоты примера, мы пока не используем порог, а полагаемся на num_merges или target_vocab_size
        # if best_score < MIN_SCORE_THRESHOLD:
        #     print(f"\nОценка лучшей биграммы ({best_score}) ниже порога. Остановка.")
        #     break

        merged_token = "".join(best_bigram)

        # Критерий остановки: если новый токен уже в словаре (такое может быть, если он был добавлен ранее)
        if merged_token in vocab:
            # Если лучший токен уже существует, удалим его из scores, чтобы найти следующий лучший
            del scores[best_bigram]
            continue # Продолжаем поиск

        # Выводим информацию о текущем слиянии
        print(f"\nИтерация {merges_count + 1}:")
        print(f"  Лучшая биграмма для слияния: '{best_bigram[0]}' + '{best_bigram[1]}' -> '{merged_token}' (Score: {best_score:.4f})")

        # Выполняем слияние в корпусе
        current_tokens = merge_tokens(current_tokens, best_bigram)

        # Добавляем новый токен в словарь
        vocab.add(merged_token)
        merges_count += 1

        print(f"  Корпус после слияния: {current_tokens}")
        print(f"  Текущий размер словаря: {len(vocab)}")

        # Критерии остановки
        if target_vocab_size is not None and len(vocab) >= target_vocab_size:
            print(f"\nДостигнут целевой размер словаря ({target_vocab_size}). Остановка.")
            break
        if num_merges is not None and merges_count >= num_merges:
            print(f"\nДостигнуто заданное количество слияний ({num_merges}). Остановка.")
            break

    print(f"\nОбучение завершено. Итоговый словарь содержит {len(vocab)} токенов.")
    return sorted(list(vocab)) # Возвращаем отсортированный список токенов

def tokenize_with_wordpiece(text, vocab):
    """
    Токенизация нового текста с использованием обученного словаря WordPiece.
    """
    print(f"\n--- Токенизация нового текста ---")
    print(f"Текст для токенизации: '{text}'")

    # Предварительная обработка текста для токенизации
    processed_text = text.lower().replace('...', '').replace('.', '')
    words = processed_text.split(' ')

    final_tokens = []
    for word in words:
        if not word: # Пропускаем пустые строки, если они возникли из-за множественных пробелов
            continue

        word_tokens = []
        remaining_word = word

        while remaining_word:
            found_match = False
            # Ищем самый длинный подтокен из словаря, который является префиксом оставшейся части слова
            for i in range(len(remaining_word), 0, -1):
                subword = remaining_word[:i]
                # Для подслов, которые не являются началом слова, в реальных WordPiece токенизаторах
                # используется префикс '##'. Здесь мы упрощаем, но можно добавить:
                # if word_tokens and i < len(remaining_word):
                #     subword_with_prefix = '##' + subword
                #     if subword_with_prefix in vocab:
                #         word_tokens.append(subword_with_prefix)
                #         remaining_word = remaining_word[i:]
                #         found_match = True
                #         break

                if subword in vocab:
                    word_tokens.append(subword)
                    remaining_word = remaining_word[i:]
                    found_match = True
                    break

            if not found_match:
                # Если не удалось найти соответствующий токен, разбиваем на символы
                # или используем токен [UNK]
                if remaining_word[0] in vocab: # Если символ есть в словаре
                    word_tokens.append(remaining_word[0])
                else: # Если символ даже не в начальном словаре (очень редкий случай)
                    word_tokens.append('[UNK]')
                remaining_word = remaining_word[1:]


        final_tokens.extend(word_tokens)
        final_tokens.append(' ') # Добавляем пробел между токенизированными словами

    # Удаляем последний пробел, если он есть
    if final_tokens and final_tokens[-1] == ' ':
        final_tokens.pop()

    print(f"Токенизированный текст: {final_tokens}")
    return final_tokens

# --- Пример использования ---
text_to_train = "В училище учитель учит ученикам по новому учебнику"

# Обучаем токенизатор, задав максимальное количество слияний для демонстрации
# В реальных сценариях задают target_vocab_size (например, 30000)
final_vocab = train_wordpiece(text_to_train, num_merges=50) # Ограничимся 10 слияниями для примера

print(f"\nИтоговый словарь WordPiece: {final_vocab}")

# Токенизируем тот же текст с использованием обученного словаря
tokenize_with_wordpiece(text_to_train, final_vocab)

# Пример токенизации нового слова, которое может быть разбито на подслова
new_text = "учительница"
# Добавим '##ница' в словарь для демонстрации, если оно не было сформировано
if 'ница' not in final_vocab:
    final_vocab.append('ница')
    final_vocab.append('##ница') # В реальных WordPiece так обозначаются подслова
    final_vocab.sort() # Пересортируем, чтобы было красиво

print(f"\nИтоговый словарь WordPiece (с 'ница' для демонстрации): {final_vocab}")
tokenize_with_wordpiece(new_text, final_vocab)

# Пример слова, которое может быть разбито на части, если целиком его нет в словаре
# Для этого нам нужно, чтобы 'учитель' был в словаре, а 'учительница' - нет,
# но 'ница' или '##ница' были.

# Давайте создадим упрощенный словарь для демонстрации токенизации подслов
simplified_vocab = set(['у', 'ч', 'и', 'т', 'е', 'л', 'ь', 'н', 'ц', 'а', 'учитель', '##ница', '##тель', '##ница', '##а'])
print(f"\n--- Демонстрация токенизации подслов с упрощенным словарем ---")
tokenize_with_wordpiece("учительница", sorted(list(simplified_vocab)))

# Униграмной языковой модели

In [ ]:
import re
from collections import Counter

def tokenize(text):
    """
    Токенизирует текст, приводя его к нижнему регистру и удаляя знаки препинания.
    Args:
        text (str): Входной текст.
    Returns:
        list: Список токенов (слов).
    """
    # Приводим текст к нижнему регистру
    text = text.lower()
    # Удаляем знаки препинания и разбиваем на слова
    # Используем re.findall для извлечения только буквенных последовательностей
    tokens = re.findall(r'\b[а-яёa-z]+\b', text)
    return tokens

def train_unigram_model(corpus_text):
    """
    Обучает униграмную языковую модель на заданном корпусе текста.
    Рассчитывает вероятности слов методом максимального правдоподобия (MLE)
    и со сглаживанием по Лапласу.
    Args:
        corpus_text (str): Текст корпуса для обучения модели.
    Returns:
        tuple: Кортеж, содержащий:
            - dict: Вероятности слов по MLE.
            - dict: Вероятности слов со сглаживанием по Лапласу.
            - int: Общее количество слов в корпусе (N).
            - int: Размер словаря (количество уникальных слов).
    """
    # Шаг 1: Токенизация корпуса
    tokens = tokenize(corpus_text)

    # Общее количество слов в корпусе (N)
    N = len(tokens)
    if N == 0:
        print("Ошибка: Корпус пуст после токенизации.")
        return {}, {}, 0, 0

    # Шаг 2: Подсчет частот слов
    word_counts = Counter(tokens)

    # Шаг 3: Определение словаря и его размера
    vocabulary = list(word_counts.keys())
    V = len(vocabulary)

    # Шаг 4: Расчет вероятностей слов (MLE)
    mle_probabilities = {}
    for word, count in word_counts.items():
        mle_probabilities[word] = count / N

    # Расчет вероятностей слов со сглаживанием по Лапласу
    laplace_probabilities = {}
    for word in vocabulary:
        laplace_probabilities[word] = (word_counts[word] + 1) / (N + V)
    # Для слов, которых нет в словаре, но могут встретиться в новом тексте
    # Их вероятность со сглаживанием будет (0 + 1) / (N + V)
    # Мы не добавляем их сюда напрямую, но это учитывается при использовании модели.

    print(f"Общее количество слов в корпусе (N): {N}")
    print(f"Размер словаря (|V|): {V}")
    print("\nЧастоты слов:")
    for word, count in word_counts.items():
        print(f"  '{word}': {count}")
    print("\nВероятности слов (MLE):")
    for word, prob in mle_probabilities.items():
        print(f"  P('{word}') = {prob:.4f}")
    print("\nВероятности слов (Laplace Smoothing):")
    for word, prob in laplace_probabilities.items():
        print(f"  P_Laplace('{word}') = {prob:.4f}")

    return mle_probabilities, laplace_probabilities, N, V

def calculate_sentence_probability(sentence, model_probabilities, N, V, smoothing_type='laplace'):
    """
    Рассчитывает вероятность предложения, используя обученную униграмную модель.
    Args:
        sentence (str): Предложение для оценки.
        model_probabilities (dict): Словарь вероятностей слов (MLE или Laplace).
        N (int): Общее количество слов в обучающем корпусе.
        V (int): Размер словаря обучающего корпуса.
        smoothing_type (str): Тип сглаживания ('none' для MLE, 'laplace' для сглаживания по Лапласу).
    Returns:
        float: Вероятность предложения.
    """
    sentence_tokens = tokenize(sentence)
    probability = 1.0

    print(f"\nОценка вероятности предложения: '{sentence}'")
    print(f"Токены предложения: {sentence_tokens}")

    for token in sentence_tokens:
        if smoothing_type == 'laplace':
            # Используем формулу Лапласа для каждого токена
            # Если токена нет в model_probabilities (т.е. Count(token) = 0),
            # то его вероятность будет (0 + 1) / (N + V)
            word_count_in_model = model_probabilities.get(token, 0) * N # Восстанавливаем Count(token)
            # Если слово не было в словаре, то word_count_in_model будет 0.0,
            # но для Лапласа нам нужен его реальный счетчик (0).
            # Проще использовать прямой расчет:
            if token in model_probabilities:
                p_token = (word_count_in_model + 1) / (N + V)
            else:
                p_token = 1 / (N + V) # Для неизвестных слов
            print(f"  P_Laplace('{token}') = {p_token:.8f}")
        else: # Без сглаживания (MLE)
            p_token = model_probabilities.get(token, 0.0) # Если слово не найдено, вероятность 0
            print(f"  P_MLE('{token}') = {p_token:.8f}")

        if p_token == 0:
            # Если вероятность слова 0, то вероятность всего предложения 0
            probability = 0.0
            print(f"  Обнаружено неизвестное слово '{token}' без сглаживания. Вероятность предложения = 0.")
            break
        probability *= p_token

    return probability

# --- Пример использования ---
corpus_text = "Ученики пишут диктант. Учитель диктует. Мы пишем, он пишет, они пишут. Пишите внимательно."

print("--- Обучение Униграмной модели ---")
mle_probs, laplace_probs, N_corpus, V_corpus = train_unigram_model(corpus_text)

# Оценка вероятности предложения из корпуса (без сглаживания)
sentence1 = "Ученики пишут диктант"
prob_sentence1_mle = calculate_sentence_probability(sentence1, mle_probs, N_corpus, V_corpus, smoothing_type='none')
print(f"\nВероятность предложения '{sentence1}' (MLE): {prob_sentence1_mle:.15f}")

# Оценка вероятности предложения из корпуса (со сглаживанием по Лапласу)
prob_sentence1_laplace = calculate_sentence_probability(sentence1, laplace_probs, N_corpus, V_corpus, smoothing_type='laplace')
print(f"\nВероятность предложения '{sentence1}' (Laplace Smoothing): {prob_sentence1_laplace:.15f}")

# Оценка вероятности нового предложения с неизвестным словом (без сглаживания)
sentence2 = "Учитель читает книгу"
prob_sentence2_mle = calculate_sentence_probability(sentence2, mle_probs, N_corpus, V_corpus, smoothing_type='none')
print(f"\nВероятность предложения '{sentence2}' (MLE): {prob_sentence2_mle:.15f}")

# Оценка вероятности нового предложения с неизвестным словом (со сглаживанием по Лапласу)
prob_sentence2_laplace = calculate_sentence_probability(sentence2, laplace_probs, N_corpus, V_corpus, smoothing_type='laplace')
print(f"\nВероятность предложения '{sentence2}' (Laplace Smoothing): {prob_sentence2_laplace:.15f}")

# Пример с одним словом, которого нет в корпусе
unknown_word_sentence = "школа"
prob_unknown_laplace = calculate_sentence_probability(unknown_word_sentence, laplace_probs, N_corpus, V_corpus, smoothing_type='laplace')
print(f"\nВероятность слова '{unknown_word_sentence}' (Laplace Smoothing): {prob_unknown_laplace:.15f}")


# SentencePiece

In [ ]:
# Импортируем библиотеку SentencePiece
import sentencepiece as spm
import os # Для работы с файловой системой

print("--- Полный пример использования SentencePiece ---")

# Шаг 1: Установка SentencePiece (инструкция)
# Если у вас еще не установлен SentencePiece, вы можете установить его с помощью pip:
# pip install sentencepiece
# (Раскомментируйте строку выше и запустите в терминале, если необходимо)

# Шаг 2: Подготовка данных для обучения
# Для обучения SentencePiece требуется текстовый файл.
# Создадим пример файла с текстом, который мы будем использовать.
# Используем тот же текст, что и в лекции.
text_data = """Учитель говорит: «Откройте учебники». Ученики открывают учебники. Откройте страницу 15. Страница сложная. Страницы учебника содержат много информации."""

# Определяем имя файла для обучения
train_file_name = "train_text_for_sp.txt"

# Создаем файл для обучения и записываем в него текст
try:
    with open(train_file_name, "w", encoding="utf-8") as f:
        f.write(text_data)
    print(f"\nСоздан файл для обучения: '{train_file_name}'")
except IOError as e:
    print(f"Ошибка при создании файла для обучения: {e}")
    exit() # Выходим, если не удалось создать файл

# Шаг 3: Обучение модели SentencePiece
# Мы будем обучать модель с использованием алгоритма Unigram (по умолчанию).
# model_prefix: префикс для файлов модели (model.model и model.vocab)
# vocab_size: желаемый размер словаря подслов
# character_coverage: процент символов, которые должны быть покрыты моделью
#                  (1.0 означает, что все символы из входных данных должны быть представлены)
# model_type: 'unigram' (по умолчанию) или 'bpe'
# input: путь к файлу(ам) для обучения

model_prefix = "my_custom_sentencepiece_model"
vocab_size = 50 # Увеличим размер словаря для лучшей демонстрации

print(f"\nНачинается обучение модели SentencePiece с vocab_size={vocab_size}...")
try:
    spm.SentencePieceTrainer.train(
        input=train_file_name,
        model_prefix=model_prefix,
        vocab_size=vocab_size,
        character_coverage=1.0,
        model_type='unigram' # Можно изменить на 'bpe'
    )
    print(f"Обучение завершено. Созданы файлы: '{model_prefix}.model' и '{model_prefix}.vocab'")
except Exception as e:
    print(f"Ошибка при обучении модели SentencePiece: {e}")
    # Попытаемся удалить созданный файл, если обучение не удалось
    if os.path.exists(train_file_name):
        os.remove(train_file_name)
    exit()

# Шаг 4: Загрузка обученной модели
# После обучения мы можем загрузить модель для использования.
sp = spm.SentencePieceProcessor()
model_path = f"{model_prefix}.model"

try:
    sp.load(model_path)
    print(f"\nМодель SentencePiece загружена: '{model_path}'")
except Exception as e:
    print(f"Ошибка при загрузке модели SentencePiece: {e}")
    # Попытаемся удалить созданные файлы, если загрузка не удалась
    if os.path.exists(train_file_name): os.remove(train_file_name)
    if os.path.exists(f"{model_prefix}.model"): os.remove(f"{model_prefix}.model")
    if os.path.exists(f"{model_prefix}.vocab"): os.remove(f"{model_prefix}.vocab")
    exit()

# Шаг 5: Токенизация текста (кодирование)
# Теперь используем загруженную модель для токенизации нашего текста.

input_text = "Учитель говорит: «Откройте учебники». Ученики открывают учебники. Откройте страницу 15. Страница сложная. Страницы учебника содержат много информации."

print(f"\n--- Токенизация ---")
print(f"Исходный текст: '{input_text}'")

# Кодирование в ID токенов
encoded_ids = sp.encode_as_ids(input_text)
print(f"Токенизация в ID: {encoded_ids}")

# Кодирование в строковые токены (подслова)
encoded_pieces = sp.encode_as_pieces(input_text)
print(f"Токенизация в подслова: {encoded_pieces}")

# Шаг 6: Детокенизация текста (декодирование)
# Восстановим исходный текст из токенизированных ID или подслов.

print(f"\n--- Детокенизация ---")

decoded_text_from_ids = sp.decode_ids(encoded_ids)
print(f"Детокенизация из ID: '{decoded_text_from_ids}'")

decoded_text_from_pieces = sp.decode_pieces(encoded_pieces)
print(f"Детокенизация из подслов: '{decoded_text_from_pieces}'")

# Проверка обратимости
if input_text == decoded_text_from_ids and input_text == decoded_text_from_pieces:
    print("\nПроцесс токенизации и детокенизации полностью обратим!")
else:
    print("\nВнимание: Детокенизированный текст не полностью совпадает с исходным.")
    print(f"Ожидалось: '{input_text}'")
    print(f"Получено из ID: '{decoded_text_from_ids}'")
    print(f"Получено из подслов: '{decoded_text_from_pieces}'")


# Шаг 7: Очистка (удаление созданных файлов)
# Удалим файлы, созданные в процессе обучения, чтобы не засорять рабочую директорию.
print(f"\n--- Очистка временных файлов ---")
try:
    if os.path.exists(train_file_name):
        os.remove(train_file_name)
        print(f"Удален файл: '{train_file_name}'")
    if os.path.exists(f"{model_prefix}.model"):
        os.remove(f"{model_prefix}.model")
        print(f"Удален файл: '{model_prefix}.model'")
    if os.path.exists(f"{model_prefix}.vocab"):
        os.remove(f"{model_prefix}.vocab")
        print(f"Удален файл: '{model_prefix}.vocab'")
except OSError as e:
    print(f"Ошибка при удалении файлов: {e}")

print("\n--- Использование SentencePiece завершено ---")
